# 🤖 Importación de librerías principales
Este notebook importa las librerías necesarias para el análisis y forecasting de ventas 2025.

In [21]:
# 🤖 Importación de librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import streamlit as st
import holidays

# 🤖 Carga de datos de ventas 2025 para inferencia
Se carga el archivo de ventas 2025 para realizar el forecasting.

In [22]:
# 🤖 Carga de datos de ventas 2025 para inferencia
inferencia_df = pd.read_csv('../data/raw/inferencia/ventas_2025_inferencia.csv')
inferencia_df.head()

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [23]:
# 🤖 Conversión de la columna fecha a tipo datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# 🤖 Creación de variables temporales usando holidays para España
es_holidays = holidays.country_holidays('ES', years=inferencia_df['fecha'].dt.year.unique())
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.dayofweek  # 0=Lunes, 6=Domingo
inferencia_df['nombre_dia_semana'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['es_fin_semana'] = inferencia_df['dia_semana'].isin([5,6])
inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(es_holidays)

# Black Friday: último viernes de noviembre
def es_black_friday(fecha):
    if fecha.month == 11:
        viernes_noviembre = pd.date_range(start=f'{fecha.year}-11-01', end=f'{fecha.year}-11-30', freq='W-FRI')
        return fecha in viernes_noviembre[-1:]
    return False

inferencia_df['es_black_Friday'] = inferencia_df['fecha'].apply(es_black_friday)

# Cyber Monday: primer lunes después de Black Friday
def es_cyber_monday(fecha):
    if fecha.month == 11 or fecha.month == 12:
        viernes_noviembre = pd.date_range(start=f'{fecha.year}-11-01', end=f'{fecha.year}-11-30', freq='W-FRI')
        if len(viernes_noviembre) > 0:
            black_friday = viernes_noviembre[-1]
            cyber_monday = black_friday + pd.Timedelta(days=3)
            return fecha == cyber_monday
    return False

inferencia_df['es_cyber_Monday'] = inferencia_df['fecha'].apply(es_cyber_monday)

# Variable: semana del año
inferencia_df['semana_año'] = inferencia_df['fecha'].dt.isocalendar().week
# Variable: trimestre
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter
# Variable: día laborable (no festivo y no fin de semana)
inferencia_df['es_laborable'] = (~inferencia_df['es_fin_semana']) & (~inferencia_df['es_festivo'])

# 🤖 Creación de lags y media móvil de 7 días
inferencia_df = inferencia_df.sort_values(['producto_id', 'año', 'fecha'])
for lag in range(1, 8):
    inferencia_df[f'lag_{lag}'] = inferencia_df.groupby(['producto_id', 'año'])['unidades_vendidas'].shift(lag)
inferencia_df['media_movil_7d'] = inferencia_df.groupby(['producto_id', 'año'])['unidades_vendidas'].transform(
    lambda x: x.rolling(window=7, min_periods=7).mean().shift(1)
)

# 🤖 Creación de variable descuento porcentaje
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100
inferencia_df['descuento_porcentaje'] = inferencia_df['descuento_porcentaje'].round(2)

# 🤖 Creación de variable precio_competencia y ratio_precio
competidores = ['Amazon', 'Decathlon', 'Deporvillage']
inferencia_df['precio_competencia'] = inferencia_df[competidores].mean(axis=1)
inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']
inferencia_df = inferencia_df.drop(columns=competidores)

# 🤖 Copia de variables categóricas y one hot encoding
inferencia_df['nombre_h'] = inferencia_df['nombre']
inferencia_df['categoria_h'] = inferencia_df['categoria']
inferencia_df['subcategoria_h'] = inferencia_df['subcategoria']

nombre_dummies = pd.get_dummies(inferencia_df['nombre_h'], prefix='nombre_h')
categoria_dummies = pd.get_dummies(inferencia_df['categoria_h'], prefix='categoria_h')
subcategoria_dummies = pd.get_dummies(inferencia_df['subcategoria_h'], prefix='subcategoria_h')

inferencia_df = pd.concat([inferencia_df, nombre_dummies, categoria_dummies, subcategoria_dummies], axis=1)

# 🤖 Eliminar registros de octubre y dejar solo noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11]

# 🤖 Guardar el dataframe transformado
inferencia_df.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)
print('Archivo guardado en data/processed/inferencia_df_transformado.csv')

Archivo guardado en data/processed/inferencia_df_transformado.csv


C:\Users\isaac\AppData\Local\Temp\ipykernel_47996\2650866479.py:12: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(es_holidays)


In [25]:
inferencia_df.columns

Index(['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria',
       'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta',
       'ingresos', 'año', 'mes', 'dia_mes', 'dia_semana', 'nombre_dia_semana',
       'es_fin_semana', 'es_festivo', 'es_black_Friday', 'es_cyber_Monday',
       'semana_año', 'trimestre', 'es_laborable', 'lag_1', 'lag_2', 'lag_3',
       'lag_4', 'lag_5', 'lag_6', 'lag_7', 'media_movil_7d',
       'descuento_porcentaje', 'precio_competencia', 'ratio_precio',
       'nombre_h', 'categoria_h', 'subcategoria_h',
       'nombre_h_Adidas Own The Run Jacket', 'nombre_h_Adidas Ultraboost 23',
       'nombre_h_Asics Gel Nimbus 25', 'nombre_h_Bowflex SelectTech 552',
       'nombre_h_Columbia Silver Ridge',
       'nombre_h_Decathlon Bandas Elásticas Set', 'nombre_h_Domyos BM900',
       'nombre_h_Domyos Kit Mancuernas 20kg',
       'nombre_h_Gaiam Premium Yoga Block', 'nombre_h_Liforme Yoga Pad',
       'nombre_h_Lotuscrafts Yoga Bolster', 'nombre_h_M